In [ ]:
import sys, os
import itertools
from pathlib import Path
ROOT_DIR=Path(os.getcwd()).parent.parent
sys.path.append(str(ROOT_DIR))

from src.config import DB_DIR
from src.data.equity_data.etf.holdings import get_ark_etf_holdings, scrape_webpage
import src.data.equity_data.etf.btc_etf_holdings as BTC_ETF_Scraper

import streamlit as st
import plotly.express as px
import sqlite3
import datetime
import pandas as pd

In [ ]:
BTC_ETF_Scraper.get_holdings()

In [ ]:
holdings_data = BTC_ETF_Scraper.scrape_btc_etf_holdings(3)
holdings_data

In [ ]:
BTC_ETF_Scraper.write_to_db(holdings_data)

In [ ]:
BTC_ETF_Scraper.correct_date('ARKB','2024-01-18','2024-01-17')

In [ ]:
import numpy as np
BTC_ETF_Scraper.insert_records('FBTC','2024-01-17',20506.54,874645041.15,np.nan,np.nan)
BTC_ETF_Scraper.insert_records('EZBC','2024-01-17',1160,49492977,np.nan,np.nan)
BTC_ETF_Scraper.insert_records('BTCO','2024-01-17',np.nan,137500000,np.nan,np.nan)

In [ ]:
BTC_ETF_Scraper.fill_missing_price(method='mv/quant')
avg_mv = BTC_ETF_Scraper.get_holdings().groupby('date').average_mv.mean().to_dict()
for date, p in avg_mv.items():
    BTC_ETF_Scraper.fill_missing_price(method='value', date=date, price=p)    
BTC_ETF_Scraper.fill_missing_holdings()
#.set_index(['date','etf_ticker']).btc_holdings.unstack()

In [ ]:
BTC_ETF_Scraper.get_holdings()

In [ ]:
fig = px.bar(
    BTC_ETF_Scraper.get_holdings().set_index(['date','etf_ticker'])\
        .btc_holdings.unstack().rename(pd.to_datetime)[['ARKB','IBIT','FBTC','BTCO','BRRR','EZBC']]\
        .dropna(how='all')\
        .rename(columns={'ARKB': 'ARK 21Shares Bitcoin ETF', 
                         'IBIT': '贝莱德比特币ETF',
                         'FBTC': 'Fidelity比特币ETF',
                         'BRRR': 'Valkyrie Bitcoin ETF',
                         'BTCO': 'Invesco Bitcoin ETF',
                         'EZBC': '富兰克林比特币ETF'}),
    barmode='stack',
    color_discrete_sequence=['#FFD300','#FF0800','#00A86B','orange','#0080FF','#AF69EF'],
    template='plotly_dark'    
)
fig.update_layout(
    height=700, width=1000,
    title=f'比特币ETF持仓量（截止 2024-1-17）',
    xaxis_title='持仓日期',
    yaxis_title='比特币持有量',
    legend_title_text=None,
    legend=dict(orientation="v",yanchor="top",y=1,xanchor="left",x=1),
    annotations=[
        dict(xref="paper",yref="paper",x=0,y=-1,showarrow=False,
            text="数据来源：ARK Invest, BlackRock, Valkyrie, Invesco, Franklin Templeton",
            font=dict(size=12,color="#ffffff"
            )
        )
    ],
)
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])